In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Link Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder


Mounted at /content/gdrive


### Try your GPU power

In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=c0b1a8b35487d9e77e63e2e056d233e0c231eb1418555b6cb72670fcdc70f215
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.5 GB  | Proc size: 500.6 MB
GPU RAM Free: 14968MB | Used: 111MB | Util   1% | Total 15079MB


In [0]:
#!kill -9 -1

## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [5]:
import numpy as np 
import sys
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
from keras.preprocessing.image import img_to_array
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from scipy.ndimage import rotate
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

Using TensorFlow backend.


**Preprocessing**

In [0]:
def data_load(directory_name): 
    filenames = os.listdir(directory_name)
    imgs = []
    for i, fileNb in enumerate(filenames):
        full_name = directory_name+fileNb
        img=mpimg.imread(full_name)
        imgr = img_to_array(img)
        imgs.append(imgr)
        sys.stdout.write("\rImage {}/{} is being loaded".format(i+1,len(filenames)))
        sys.stdout.flush()    
    return np.asarray(imgs)

def rot(image, xy, angle):
    im_rot = rotate(image,angle, reshape=False) 
    org_center = (np.array(image.shape[:2][::-1])-1)/2.
    rot_center = (np.array(im_rot.shape[:2][::-1])-1)/2.
    org = xy-org_center
    a = np.deg2rad(angle)
    new = np.array([org[0]*np.cos(a) + org[1]*np.sin(a),
            -org[0]*np.sin(a) + org[1]*np.cos(a) ])
    return im_rot

def image_generator(imgs, labels, new_shape, patch_size, batch_size = 1, upsample=False):
    np.random.seed(0)

    width_img, height_img, _ = imgs[0].shape
    
    padding = new_shape//2
    paddedImages = []
    for i in imgs:
        paddedImages.append(np.lib.pad(i, ((padding,padding),(padding,padding),(0,0)), 'reflect'))
        
    while True:
        batch_input = []
        batch_output = [] 
        
        ind = np.random.randint(0, len(imgs))  
        img = paddedImages[ind]
        gt  = labels[ind]
        
        angls = [0, 360, 15]
        zooms = [1., 0.85, 0.8, 0.75, 0.8, 0.85,
                 1., 0.85, 0.8, 0.75, 0.8, 0.85,
                 1., 0.85, 0.8, 0.75, 0.8, 0.85,
                 1., 0.85, 0.8, 0.75, 0.8, 0.85]
 
        for angle in range(len(angls)):
            img = rot(img, np.array([width_img+2*padding, height_img+2*padding]), angls[angle])
            gt = rot(gt, np.array([width_img, height_img]), angls[angle]) 

        background_count = 0
        road_count = 0
        while len(batch_input) < batch_size:
            x = np.empty((new_shape, new_shape, 3))
            y = np.empty((new_shape, new_shape, 3))
            
            boundary = int((width_img - width_img / np.sqrt(2)) / 2)

            center_x = np.random.randint(patch_size//2 + boundary, width_img-boundary-patch_size//2)
            center_y = np.random.randint(patch_size//2 + boundary, height_img-boundary-patch_size//2)
            
            x = img[center_x - patch_size//2:center_x + patch_size//2 + 2 * padding,
                    center_y - patch_size//2:center_y + patch_size//2 + 2 * padding]
            y =  gt[center_x - patch_size//2 : center_x + patch_size//2,
                    center_y - patch_size//2 : center_y + patch_size//2]
            
            # vertical
            if(np.random.randint(0, 2)):
                x = np.flipud(x)
            
            # horizontal
            if(np.random.randint(0, 2)):
                x = np.fliplr(x)
            
            label = [0., 1.] if (np.array([np.mean(y)]) >  0.25) else [1., 0.]
            
            # makes sure we have an even distribution of road and non road if we oversample
            if not upsample:
                batch_input.append(x)
                batch_output.append(label)
            elif label == [1.,0.]: # case background
                background_count += 1
                if background_count != batch_size // 2:
                    batch_input.append(x)
                    batch_output.append(label)
            elif label == [0.,1.]: # case road
                road_count += 1
                if road_count != batch_size // 2:
                    batch_input.append(x)
                    batch_output.append(label)
                
        batch_x = np.array(batch_input)
        batch_y = np.array(batch_output)


        yield(batch_x, batch_y) 

# Reshaping images to 256
def reshape(imgs, input_shape, target_size = (256,256)):
    out=np.zeros(input_shape)
    A=0
    for i in imgs:
        img = i/ 255
        img = trans.resize(img,target_size)
        img = np.reshape(img,(1,)+img.shape)
        out[A, :,:,:] = img
        A += 1
    return out

**Loading the data**

In [7]:
imgs = data_load(root_path+'Data/training/images/')
gts = data_load(root_path+'Data/training/groundtruth/')

imgs2 = reshape(imgs, input_shape = (100,256,256,3))
gts2  = reshape(gts, input_shape = (100,256,256,3))

TEST_SIZE = 0.2
Xtrain, Xtest, gt_train, gt_test = train_test_split(imgs2, gts2, test_size=TEST_SIZE, random_state=1)


Image 100/100 is being loaded

**Image generators**

In [0]:
train_generator      = image_generator(Xtrain, gt_train, 64, 32)
validation_generator = image_generator(Xtest, gt_test, 64, 32)

**Defining and training the model**

In [0]:
def uneti(input_size, pretrained_weights=None):
    inputs = Input(shape=input_size)

    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Flatten()(conv9)
    conv10 = Dense(2, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

In [0]:
model = uneti(input_size = (96,96,3), pretrained_weights=None)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 96, 96, 64)   1792        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 96, 96, 64)   36928       conv2d_24[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 48, 48, 64)   0           conv2d_25[0][0]                  
____________________________________________________________________________________________

In [0]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=2000,
                    epochs=1, 
                    verbose=1,
                    validation_data = validation_generator,
                    validation_steps = 2000
                    )

Epoch 1/1
2000/2000 [==============================] - 897s 449ms/step - loss: 1.0960e-07 - acc: 1.0000 - val_loss: 1.0960e-07 - val_acc: 1.0000


**Saving the model**

In [0]:
# serialize model to JSON
OUTPUT_FILENAME = "unet_windowed"
model_json = model.to_json()
with open(OUTPUT_FILENAME + ".json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights(OUTPUT_FILENAME + ".h5")
print("Saved model to disk")

**Load the model**

In [8]:
from keras.models import model_from_json
json_file = open(root_path+"unet_windowed.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




**Predictions : Useful functions**

In [0]:
def create_window(im, window_size, patch_size=32, batch_size=1):
    print("image", im.shape)
    list_patches = []
    is_2d = len(im.shape) < 3
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    padSize = (window_size-patch_size)//2
    is_2d = len(im.shape) < 3
    paddedImages = []
    for i in im:
        if is_2d:
          paddedImages.append(np.lib.pad(i,((padSize,padSize),(padSize,padSize)),'reflect'))
        else:
          paddedImages.append(np.lib.pad(i,((padSize,padSize),(padSize,padSize)),'reflect'))
    batch_input=[]
    while len(batch_input) < batch_size:
            x = np.empty((window_size, window_size, 3))
            
            boundary = int((imgwidth - imgwidth / np.sqrt(2)) / 2)

            center_x = np.random.randint(patch_size//2 + boundary, imgwidth-boundary-patch_size//2)
            center_y = np.random.randint(patch_size//2 + boundary, imgheight-boundary-patch_size//2)

            x = im[center_x - patch_size//2:center_x + patch_size//2 + 2 * padSize,
                    center_y - patch_size//2:center_y + patch_size//2 + 2 * padSize]
            batch_input.append(x)
    
    return np.array(batch_input)
      #padded = np.lib.pad(i, ((padSize,padSize),(padSize,padSize),(0,0)), 'reflect')

    #for i in range(padSize, imgheight + padSize, 32):
    #    for j in range(padSize,imgwidth + padSize, 32):
    #        if is_2d:
    #            im_patch = padded[j-padSize:j+32+padSize, i-padSize:i+32+padSize]
    #        else:
    #            im_patch = padded[j-padSize:j+32+padSize, i-padSize:i+32+padSize, :]
    #        list_patches.append(im_patch)
    print(paddedImages[0].shape)
    return paddedImages

def label_to_img(imgwidth, imgheight, w, h, labels):
    array_labels = np.zeros([imgwidth, imgheight])
    print('array', array_labels.shape)
    idx = 0
    print("labels", labels.shape)
    for i in range(0,imgheight,h):
        print('i', i)
        for j in range(0,imgwidth,w):
          print('j',j)
          if labels[idx][0] > 0.5: 
              l = 1
          else:
              l = 0
          array_labels[j:j+w, i:i+h] = l
          idx = idx + 1
          print("idx", idx)
    return array_labels

# Get prediction for given input image 
def get_prediction(img, model, window_size):
    data = np.asarray(test_generator(img, window_size))
    print("data", data.shape)
    output_prediction = model.predict(data)
    print("output_prediction", output_prediction.shape)
    img_prediction = label_to_img(data.shape[0], data.shape[1], 32, 32, output_prediction)
    return img_prediction


# Get a concatenation of the prediction and image for given input file
def get_prediction_with_mask(img, model, window_size):
    img_prediction = get_prediction(img, model, window_size)
    cimg = concatenate_images(img, img_prediction)
    return cimg


# Get prediction overlaid on the original image for given input file
def get_prediction_with_overlay(img, model, window_size):
    img_prediction = get_prediction(img,model, window_size)
    oimg = make_img_overlay(img, img_prediction)
    return oimg

def generate_images(model, imgs, gt_imgs):
        prediction_training_dir = root_path+"predictions_training_unet/"
        #if not os.path.isdir(prediction_training_dir):
        #    os.mkdir(prediction_training_dir)
        
        for i in range(1, 100):
          pimg = get_prediction_with_mask(imgs[i-1], model, WINDOW_SIZE)
          Image.fromarray(pimg).save(prediction_training_dir + "prediction_" + str(i) + ".png")
          oimg = get_prediction_with_overlay(imgs[i-1], model, WINDOW_SIZE)
          oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")
        
        checkImageTrainSet(model, imgs, gt_imgs, WINDOW_SIZE)     

def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

In [0]:
def test_generator(imgs, new_shape, batch_size = 1):

    width_img, height_img = imgs[0].shape
    
    padding = new_shape//2
    paddedImages = []
    for i in imgs:
        paddedImages.append(np.lib.pad(i, ((padding,padding),(padding,padding)), 'reflect'))
        

    return paddedImages

In [0]:
WINDOW_SIZE = 64



**Predictions**

In [0]:
test = imgs
test_reshaped = reshape(test, input_shape=(100,256,256,3))

In [27]:
predictions=[]
for K in test:
  prediction = get_prediction(K, model, 96)
  predictions.append(prediction)

generate_images(model, test_reshaped, predictions)

data (400, 496, 99)


ValueError: ignored